# Extraction Utilities
> Contains helpful functions for extracting embeddings and preparing data for it.

In [ ]:
# default_exp utils

In [ ]:
# Create a format for saving the embeddings and loading it back in.

In [ ]:
#export
from fastai.tabular.all import *
from copy import deepcopy
import json
import ast

We'll create a fastai learner, and extract embeddings from it. But it will be possible to do so for any pytorch model.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race','sex'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

We'll need to create a metadata dictionary for the source data which contains all the categories, and the classes for each category in the given format.

In [ ]:
meta = {
    "categories":['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    "workclass": {
        "classes": ['nan', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov',
           ' Federal-gov', ' Local-gov', ' ?', ' Without-pay',
           ' Never-worked'],
    },
    'education': {
        "classes": ['nan', ' Assoc-acdm', ' Masters', ' HS-grad', ' Prof-school', ' 7th-8th',
       ' Some-college', ' 11th', ' Bachelors', ' Assoc-voc', ' 10th',
       ' 9th', ' Doctorate', ' 12th', ' 1st-4th', ' 5th-6th',
       ' Preschool']
    },
    "marital-status": {
        "classes": ['nan', ' Married-civ-spouse', ' Divorced', ' Never-married', ' Widowed',
       ' Married-spouse-absent', ' Separated', ' Married-AF-spouse']
    },
    "occupation": {
        "classes": ["nan", ' Exec-managerial', ' Prof-specialty', ' Other-service',
       ' Handlers-cleaners', ' Craft-repair', ' Adm-clerical', ' Sales',
       ' Machine-op-inspct', ' Transport-moving', ' ?',
       ' Farming-fishing', ' Tech-support', ' Protective-serv',
       ' Priv-house-serv', ' Armed-Forces']
    },
    "relationship": {
        "classes": ['nan', ' Wife', ' Not-in-family', ' Unmarried', ' Husband', ' Own-child',
       ' Other-relative']
    },
    "race": {
        "classes": ['nan', ' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other']
    }
}

In [ ]:
#export

#Meta Format: {'categories':['cat1','cat2'......],'cat1':{'classes':['class1','class2'......]},'cat2':...........}
def extract_meta_from_learner(learner):
    cat_list=[]
    for i in learner.dls.categorify.classes:
        cat_list.append(i)
    temp_meta=dict.fromkeys(cat_list)
    t=learner.dls.categorify.classes
    meta={"categories":cat_list}
    for i in t:
        temp_meta[i]={"classes":list(t[i])}
    meta.update(temp_meta)
    return meta

def extract_meta_from_df(df: pd.DataFrame):
    columns = [x for x in df.columns]
    cat_list = []
    for i,j in enumerate(df.dtypes):
        if j == 'object':
            cat_list.append(columns[i])
    cat_dict = dict.fromkeys(cat_list)
    meta={'categories':cat_list}
    for i in cat_dict:
        cat_dict[i] = {}
        cat_dict[i]["classes"] = list(df[i].unique())
    meta.update(cat_dict)
    return meta

In [ ]:
#export
class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)

In [ ]:
#export
def extractembeds(model, embeddinglg: str, metadict, path):
    '''
    model: Any pytorch model, containing a layergroup with all the embedding layers.
    embeddinglg: Name of the layer group containing the embedding layers.
    metadict: A dictionary containing relevant metadata. Check the format given in docs for further details.
    path: Path of the json 
    '''
    embedsdict = deepcopy(metadict)
    for i, cat in enumerate(metadict['categories']):
        try:
            classes = metadict[cat]['classes']
            layer = getattr(model, embeddinglg)[i]
            assert (layer.num_embeddings == len(classes)), f"embeddings dimension {layer.num_embeddings} !=  num of classes {len(classes)} for vairable {cat}. Embeddings should have same number of classes. Something might have gone wrong."
            embedsdict[cat]["embeddings"] = layer.weight.cpu().detach().numpy().tolist()
        except KeyError:
            pass
    with open(path, 'w') as fp:
        json.dump(embedsdict, fp,cls=CustomJSONizer)
    return embedsdict

In [ ]:
meta_new=extract_meta_from_learner(learn)
embeddict = extractembeds(learn.model, "embeds", meta_new, "test.json")

In [ ]:
embeddict['categories']

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'education-num_na']

In [ ]:
#hide
assert len(embeddict['education-num_na']['classes']) == 3, "The number of classes are incorrect"
assert embeddict['categories'] == ['workclass',
				   'education',
				   'marital-status',
				   'occupation',
				   'relationship',
				   'race',
				   'sex',
				   'education-num_na'], "The number of categories in incorrect"
assert np.shape(embeddict['education-num_na']['embeddings']) == (3, 3), "Incorrect shape of embeddings"

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_ExractUtils.ipynb.
Converted 01_transfer.ipynb.
Converted 02_load_tests.ipynb.
Converted index.ipynb.
